In [ ]:
from DeepLearning.PPO import MaskablePPO
from DeepLearning.Thesis.Environments.HyperparameterTesting import HyperParamVpEndEnv
from DeepLearning.GetActionMask import getActionMask
from DeepLearning.Thesis.Observations.get_observation import getObservation, lowerBound, upperBound
import os

env = HyperParamVpEndEnv(lowerBounds=lowerBound, upperBounds=upperBound, getObservationFunction=getObservation, getActionMaskFunction=getActionMask)
actionMask = getActionMask
observation = getObservation

netArchDict = dict(pi=[128, 128, 128], vf=[128, 128, 128])
gamma = 0.99


saveName = "NumSteps_8192"
savePath = f"DeepLearning/Thesis/Hyperparameters/Models/{saveName}"

model = MaskablePPO("MlpPolicy", env, verbose=1, policy_kwargs=dict(net_arch=netArchDict), gamma=gamma, n_steps=8192, getActionMask=actionMask, getObservation=observation, savePath=savePath, tensorboard_log="./tensorboard_logs_thesis/")
model.learn(total_timesteps=1_000_000, tb_log_name=saveName)

In [ ]:
"""
Running Agent simulations
"""
from Agents.AgentRandom2 import AgentRandom2
from Agents.AgentMCTS import AgentMCTS
from Agents.AgentUCT import AgentUCT
from Agents.AgentModel import AgentMultiModel, AgentModel
from Game.CatanGame import *
from CatanSimulator import CreateGame
from DeepLearning.PPO import MaskablePPO
from Game.CatanPlayer import PlayerStatsTracker
from tabulate import tabulate
from DeepLearning.Stats import headers
import dill as pickle
from CatanData.GameStateViewer import SaveGameStateImage, DisplayImage
import math
from DeepLearning.Environments.CatanEnv import CatanEnv, SelfPlayDistribution, CatanTradingEnv, SelfPlayDistTradingEnv

from DeepLearning.Thesis.Observations.nodes_v_hexes import getNodeInHexObs, nodeInHexLowerBound, nodeInHexUpperBound

winner = [0,0,0,0]
player0Stats = PlayerStatsTracker()
Player0LosingStats = PlayerStatsTracker()
player1Stats = PlayerStatsTracker()


testModel = MaskablePPO.load("DeepLearning/Thesis/Hyperparameters/Models/NumSteps_8192/Final.zip")
# testModel2 = MaskablePPO.load("DeepLearning/Thesis/Hyperparameters/Models/NetworkArch_128_128/Final.zip")
# testModel3 = MaskablePPO.load("DeepLearning/Thesis/Hyperparameters/Models/NetworkArch_256_256/Final.zip")
# testModel4 = MaskablePPO.load("DeepLearning/Thesis/Hyperparameters/Models/NetworkArch_512_512/Final.zip")

players = [ AgentModel("P0", 0, recordStats=True, playerTrading=False, model=testModel),
            # AgentModel("P1", 1, recordStats=True, playerTrading=False, model=testModel2),
            AgentRandom2("P1", 1, recordStats=True, playerTrading=False),
            AgentRandom2("P2", 2, recordStats=True, playerTrading=False),
            AgentRandom2("P3", 3, recordStats=True, playerTrading=False),]


COLLECT_STATS = True
for episode in range(2000):
    game = CreateGame(players)
    game = pickle.loads(pickle.dumps(game, -1))
    numTurns = 0
    while True:
        currPlayer = game.gameState.players[game.gameState.currPlayer]

        agentAction = currPlayer.DoMove(game)
        agentAction.ApplyAction(game.gameState)

        if currPlayer.seatNumber == 0 and agentAction.type == 'EndTurn':
        #     DisplayImage(game.gameState, agentAction)
        #     time.sleep(1)
            numTurns += 1

        if game.gameState.currState == "OVER":
            # DisplayImage(game.gameState, agentAction)
            break
    
    # print("Winner: ", game.gameState.winner)
    winner[game.gameState.winner] += 1
    lost = game.gameState.winner != 0

    # Stats
    if COLLECT_STATS:
        game.gameState.players[0].generatePlayerStats()
        game.gameState.players[1].generatePlayerStats()

        player0Stats += game.gameState.players[0].stats
        player1Stats += game.gameState.players[1].stats
        if lost:
            Player0LosingStats += game.gameState.players[0].stats

# Collect stats
if COLLECT_STATS:
    player0Stats.getAverages()
    Player0LosingStats.getAverages()
    player1Stats.getAverages()
    player0Data = player0Stats.getList()
    player0LosingData = Player0LosingStats.getList()
    player1Data = player1Stats.getList()

    p_hat0 = winner[0] / sum(winner)
    p_hat1 = winner[1] / sum(winner)
    margin_error0 = round(100*(1.96 * math.sqrt((p_hat0 * (1 - p_hat0)) / sum(winner))), 2)
    margin_error1 = round(100*(1.96 * math.sqrt((p_hat1 * (1 - p_hat1)) / sum(winner))), 2)
    player0Data.insert(0, margin_error0)
    player0LosingData.insert(0, -1)
    player1Data.insert(0, margin_error1)
    player0Data.insert(0, winner[0]/sum(winner))
    player0LosingData.insert(0, -1)
    player1Data.insert(0, winner[1]/sum(winner))
    player0Data.insert(0, "Player0")
    player0LosingData.insert(0, "Player0LossesStats")
    player1Data.insert(0, "Player1")

    table = tabulate([player0Data, player0LosingData, player1Data], headers=headers, tablefmt='simple')
    print(table)

print(f"\nNum turns: {numTurns}")

print("\n\nWinnings: ", winner)


# Brick, ore, wool, wheat, wood

In [ ]:
import pandas as pd

# Save to csv
fileName = f'NumSteps_8192_v_3Random.csv'
df = pd.DataFrame([player0Data, player0LosingData, player1Data], columns=headers)
df.to_csv(f'DeepLearning/Thesis/Hyperparameters/Data/{fileName}', index=False)